In [6]:
import os
import numpy as np
import pandas as pd
import pickle
from timeit import default_timer as timer
from tqdm import tqdm
import matplotlib.pyplot as plt

from scipy.spatial.distance import jensenshannon
from sklearn.metrics import mean_squared_error, r2_score

import tensorflow as tf
from keras import backend as K
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))
K.clear_session()

from keras import Input, optimizers, layers, losses
from keras.models import Model
from keras.layers import Dense, Activation, Input, BatchNormalization, concatenate, ReLU, LeakyReLU, Dropout
from keras.callbacks import CSVLogger, EarlyStopping

In [7]:
data_train=pd.read_pickle('./pickle/PF_decay_train_reduced_stratified_no_log.pkl')
data_val=pd.read_pickle('./pickle/PF_decay_val_reduced_stratified_no_log.pkl')
data_mean_std=pd.read_csv('./data_mean_std.csv', index_col=0)

In [8]:
list_input=['c_tilde', 'abs_grad_c_tilde', 'c_var', 'mag_vel', 'mag_grad_vel', 'UP_delta', 'mag_strain', 'mag_vor', 'kappa_LES', 'a_T_LES']

X = data_train[list_input]
C = data_train[['Delta_del_th']]
Y = data_train[['omega_LES']]

X_val=data_val[list_input]
C_val=data_val[['Delta_del_th']]
Y_val=data_val[['omega_LES']]

dataX_row_count = X.shape[0]
dataX_column_count = len(X.columns)
dataY_column_count = len(Y.columns)
noise_size = 1
condition_size = 1

lr_disc=1e-4
lr_gen=1e-4

In [9]:
class LSGAN():
    def __init__(self):                   
        self.generator = self.build_generator()
        self.discriminator = self.build_discriminator()

        noise     = Input(shape=(noise_size,))
        x_arr     = Input(shape=(dataX_column_count,))
        condition = Input(shape=(condition_size,))

        fake_arr = self.generator([noise, condition, x_arr])
        valid = self.discriminator([x_arr, condition, fake_arr])
       
        self.discriminator.compile(optimizer=optimizers.Adam(learning_rate=lr_disc, beta_1=0.5, beta_2=0.999),
                                   loss=['mse'])

        self.discriminator.trainable = False

        self.combined = Model([noise, condition, x_arr], valid)
        self.combined.compile(optimizer=optimizers.Adam(learning_rate=lr_gen, beta_1=0.5, beta_2=0.999),
                              loss=['mse'])

    def element(self, input_tensor, n_nodes, activation):
        x = Dense(n_nodes)(input_tensor)
        # x = BatchNormalization()(x)

        if activation==0:
            x = ReLU()(x)
        elif activation==1:
            x = LeakyReLU()(x)

        return x

    def res_block(self, input_tensor, n_nodes, activation):
        x = self.element(input_tensor, n_nodes, activation)
        x = self.element(x, n_nodes, activation)
        x = self.element(x, n_nodes, activation)
    
        x = layers.add([x, input_tensor])
        if activation==0:
            x = ReLU()(x)
        elif activation==1:
            x = LeakyReLU()(x)
    
        return x

    def build_generator(self,
        n_nodes=100,
        n_res_block_gen=3):

        input_x  = Input(shape=(dataX_column_count,))
        output_x = self.element(input_x, n_nodes, 0)
        output_x = self.element(output_x, n_nodes, 0)

        input_noise  = Input(shape=(noise_size,))
        output_noise = self.element(input_noise, 25, 0)
        output_noise = self.element(output_noise, 25, 0)

        input_condition  = Input(shape=(condition_size,))
        output_condition = self.element(input_condition, 25, 0)
        output_condition = self.element(output_condition, 25, 0)

        x = concatenate([output_x, output_noise, output_condition])
        x = self.element(x, n_nodes, 0)
        for _ in range(n_res_block_gen):
            x = self.res_block(x, n_nodes, 0)

        output = Dense(dataY_column_count, activation='linear')(x)

        model = Model(inputs=[input_noise, input_condition, input_x], outputs=output)

        return(model)

    def build_discriminator(self,
        n_nodes=100,
        n_res_block_crit=3):

        input_x = Input(shape=(dataX_column_count,))
        input_condition  = Input(shape=(condition_size,))
        input_label = Input(shape=(dataY_column_count,))

        x = concatenate([input_x, input_condition, input_label])
                
        x = self.element(x, n_nodes, 1)
        for _ in range(n_res_block_crit):
            x = self.res_block(x, n_nodes, 1)

        loss = Dense(1)(x)

        model = Model(inputs=[input_x, input_condition, input_label], outputs=loss)
    
        return(model)

    def index_sweep(self, idx, size, tot):
        start=idx*size
        end=start+size

        if idx==tot:
            end=tot

        return(range(start,end))

    def predict(self, xtest, ctest):
        noise = np.random.normal(0, 1, (xtest.shape[0], noise_size))
        ypred = self.generator.predict([noise, ctest, xtest])
        return ypred
    
    def inverse_log(self, val, mean, std):
        return np.exp(val * std + mean)

    def inverse(self, val, mean, std):
        return val * std + mean

    def train(self, xtrain, ctrain, ytrain, epochs, batch_size, verbose=True, show=True, metric_val=True):
        valid = np.ones((batch_size, 1))
        fake  = np.zeros((batch_size, 1))

        dLossReal = np.zeros([epochs, 1])
        dLossFake = np.zeros([epochs, 1])
        dLoss = np.zeros([epochs, 1])
        gLoss = np.zeros([epochs, 1])

        mse_val = np.zeros([epochs])
        r2_val = np.zeros([epochs])
        js_val = np.zeros([epochs])

        mse_test = np.zeros([epochs])
        r2_test = np.zeros([epochs])
        js_test = np.zeros([epochs])

        header=pd.DataFrame(np.array([['epoch', 'd_loss_real', 'd_loss_fake', 'g_loss', 'mse_val', 'r2_val', 'js_val', 'mse_test', 'r2_test', 'js_test']]))
        header.to_csv('./history.csv', mode='w', header=False, index=False)

        Y_val_inverse = self.inverse(Y_val, data_mean_std.loc['omega_LES']['mean'], data_mean_std.loc['omega_LES']['std'])

        for epoch in range(epochs):
            t0=timer()
            no_batch=int(xtrain.shape[0] // batch_size)

            for batch_idx in tqdm(range(no_batch)):
                idx_sweep = self.index_sweep(batch_idx,batch_size,no_batch)
                idx_ran = np.random.randint(0, xtrain.shape[0], batch_size)
                
                x, c, true_labels = xtrain.iloc[idx_sweep], ctrain.iloc[idx_sweep], ytrain.iloc[idx_sweep]
                noise = np.random.normal(0, 1, (batch_size, noise_size))
                
                fake_labels = self.generator.predict([noise, c, x])
                
                d_lossreal = self.discriminator.train_on_batch([x, c, true_labels], valid)
                d_lossfake = self.discriminator.train_on_batch([x, c, fake_labels], fake)
                d_loss = 0.5 * np.add(d_lossreal, d_lossfake)

                g_loss = self.combined.train_on_batch([noise, c, x], valid)

            dLossReal[epoch,0] = d_lossreal
            dLossFake[epoch,0] = d_lossfake
            dLoss[epoch,0] = d_loss
            gLoss[epoch,0] = g_loss
        
            t1=timer()

            if verbose:
                print('\n')
                print(f'Epoch: {epoch} / dLoss: {d_loss} / gLoss: {g_loss} / Time: {t1-t0} sec')
        
            if show:
                n=np.unique(data_val['n_Delta']).shape[0]
                fig, axes = plt.subplots(2, n, figsize=(25,10), sharey=True)

                for i in range(n):
                    Delta = np.unique(data_val['n_Delta'])[i]
                    X_pred = X_val[data_val['n_Delta']==Delta].sample(n=1000, random_state=0)                    
                    C_pred = C_val[data_val['n_Delta']==Delta].sample(n=1000, random_state=0)
                    Y_true = Y_val[data_val['n_Delta']==Delta].sample(n=1000, random_state=0)

                    Y_pred = self.predict(X_pred, C_pred)                    

                    axes[0,i].scatter(self.inverse(X_pred['c_tilde'], data_mean_std.loc['c_tilde']['mean'], data_mean_std.loc['c_tilde']['std']),
                                      self.inverse(Y_true, data_mean_std.loc['omega_LES']['mean'], data_mean_std.loc['omega_LES']['std']),
                                      s=1.5)

                    axes[0,i].scatter(self.inverse(X_pred['c_tilde'], data_mean_std.loc['c_tilde']['mean'], data_mean_std.loc['c_tilde']['std']),
                                      self.inverse(Y_pred, data_mean_std.loc['omega_LES']['mean'], data_mean_std.loc['omega_LES']['std']),
                                      s=1.5)

                    axes[1,i].scatter(self.inverse(Y_true, data_mean_std.loc['omega_LES']['mean'], data_mean_std.loc['omega_LES']['std']),
                                      self.inverse(Y_pred, data_mean_std.loc['omega_LES']['mean'], data_mean_std.loc['omega_LES']['std']),
                                      s=1.5)

                plt.savefig(f'Inf_Val_Epoch_{epoch}.jpg', dpi=200, box_inches='tight')

            if metric_val:
                y_pred_val=self.predict(X_val, C_val)
                y_pred_val=self.inverse(y_pred_val, data_mean_std.loc['omega_LES']['mean'], data_mean_std.loc['omega_LES']['std'])
                y_pred_val[y_pred_val<0]=1e-8

                mse_val[epoch]=mean_squared_error(y_pred_val, Y_val_inverse)
                r2_val[epoch]=r2_score(y_pred_val, Y_val_inverse)
                js_val[epoch]=jensenshannon(y_pred_val, Y_val_inverse)

                print('\n')
                print('MSE_Val: {}, R2_Val: {}, JS_Val: {}'.format(mse_val[epoch], r2_val[epoch], js_val[epoch]))

                history=pd.DataFrame(np.array([[epoch, dLossReal[epoch,0], dLossFake[epoch,0], gLoss[epoch,0], mse_val[epoch], r2_val[epoch], js_val[epoch]]]))
                history.to_csv('./history.csv', mode='a', header=False, index=False)

        return dLossReal, dLossFake, dLoss, gLoss, mse_val, r2_val, js_val

In [ ]:
lsgan = LSGAN()
dLossReal, dLossFake, dLoss, gLoss, mse_val, r2_val, js_val = lsgan.train(X, C, Y, 100, 2**10, metric_val=True)